In [ ]:
import cv2
import numpy as np
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

%matplotlib inline


In [ ]:
def calibrateCamera():
    calChessboardPattern = (6, 6)
    calImgsPath = glob.glob("./ccalib-test-images-2/720x1280/*.jpg")
    calImgSize = (1280, 720)
    
    objPoints = []
    imgPoints = []
    
    objPt = np.zeros((calChessboardPattern[0] * calChessboardPattern[1], 3), np.float32)
    objPt[:, :2] = np.mgrid[0:calChessboardPattern[0], 0:calChessboardPattern[1]].T.reshape(-1, 2)
    
    
    w = 20
    h = 20
    fig = plt.figure(figsize=(16, 16))
    columns = 3
    rows = 5
    i = 1
    
    for image in calImgsPath:
        calImg = cv2.imread(image)
        ret, corners = cv2.findChessboardCorners(calImg, calChessboardPattern)
        
        if ret:
            imgPoints.append(corners)
            objPoints.append(objPt)
            cv2.drawChessboardCorners(calImg, calChessboardPattern, corners, ret)
            if i <= rows * columns:
                fig.add_subplot(rows, columns, i)
                plt.imshow(calImg)
                i += 1

    plt.show()
    
        

In [ ]:
calibrateCamera()